In [42]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('coba2') \
    .getOrCreate()

22/12/11 15:35:57 WARN Utils: Your hostname, beast resolves to a loopback address: 127.0.1.1; using 192.168.0.125 instead (on interface wlp4s0)
22/12/11 15:35:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/11 15:35:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/11 15:35:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# spark.stop()
spark

In [5]:
parqDF = spark.read.parquet("fhv_tripdata_2021-02.parquet")

In [6]:
# Data overview
parqDF.printSchema()
parqDF.limit(10).toPandas()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



/home/jimbo/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/home/jimbo/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 07:01:00,2021-02-01 08:33:00,NaN,NaN,NaN,B00014
1,B00021,2021-02-01 07:55:40,2021-02-01 08:06:20,173.0,82.0,NaN,B00021
2,B00021,2021-02-01 07:14:03,2021-02-01 07:28:37,173.0,56.0,NaN,B00021
3,B00021,2021-02-01 07:27:48,2021-02-01 07:35:45,82.0,129.0,NaN,B00021
4,B00037,2021-02-01 07:12:50,2021-02-01 07:26:38,NaN,225.0,NaN,B00037
5,B00037,2021-02-01 07:00:37,2021-02-01 07:09:35,NaN,61.0,NaN,B00037
6,B00112,2021-02-01 07:30:25,2021-02-01 07:57:23,NaN,26.0,NaN,B00112
7,B00149,2021-02-01 07:43:16,2021-02-01 08:03:16,NaN,72.0,NaN,B00149
8,B00221,2021-02-01 07:20:45,2021-02-01 07:21:15,NaN,244.0,NaN,B00221
9,B00225,2021-02-01 07:23:27,2021-02-01 07:55:46,NaN,169.0,NaN,B00225


In [16]:
# 1. How many taxi trips were there on February 15

parqDF.createOrReplaceTempView("Table1")
df = spark.sql("select date(pickup_datetime) as Pickup_date, count(pickup_datetime) as Trips_on_Feb15 from Table1 where pickup_datetime like '2021-02-15%' group by date(pickup_datetime)")
df.toPandas()

# df.coalesce(1).write.format("test.csv").option("header", "true").save("question1.csv")

,Pickup_date,Trips_on_Feb15
0,2021-02-15,35709


In [24]:
df.repartition(1).write.mode('default').option("header", True).csv("question1.csv")

In [25]:
# 2. Find the longest trip for each day?

parqDF.createOrReplaceTempView("Table2")
df = spark.sql("select date(pickup_datetime) as Date, max(datediff(day, pickup_datetime, dropOff_datetime)) AS Longest_Trip_per_Day from Table2 group by date(pickup_datetime) order by date(pickup_datetime) asc")
df.toPandas()

,Date,Longest_Trip_per_Day
0,2021-02-01,32
1,2021-02-02,0
2,2021-02-03,0
3,2021-02-04,27
4,2021-02-05,77
5,2021-02-06,1
6,2021-02-07,0
7,2021-02-08,6
8,2021-02-09,1
9,2021-02-10,0


In [26]:
df.repartition(1).write.mode('default').option("header", True).csv("question2.csv")

In [27]:
# 3. Find top 5 Most Frequent 'dispatching_base_num'

parqDF.createOrReplaceTempView("Table3")
df = spark.sql("select dispatching_base_num, count(dispatching_base_num) as frequency from Table3 group by dispatching_base_num order by frequency desc limit 5")
df.toPandas()

,dispatching_base_num,frequency
0,B00856,35077
1,B01312,33089
2,B01145,31114
3,B02794,30397
4,B03016,29794


In [28]:
df.repartition(1).write.mode('default').option("header", True).csv("question3.csv")

In [29]:
# 4. Find top 5 Most common location pairs (PUlocationID, DOlocationID)

parqDF.createOrReplaceTempView("Table4")
df = spark.sql("select pulocationid, dolocationid, count(concat(PUlocationID, DOlocationID)) as freq_together from Table4 where not(pulocationid = dolocationid) group by pulocationid, dolocationid order by freq_together desc limit 5")
df.toPandas()

,pulocationid,dolocationid,freq_together
0,221.0,206.0,2112
1,206.0,221.0,1309
2,221.0,115.0,1010
3,115.0,221.0,793
4,115.0,206.0,769


In [30]:
df.repartition(1).write.mode('default').option("header", True).csv("question4.csv")

In [31]:
from google.cloud import bigquery
import os

In [32]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"/home/jimbo/.google/credentials/google_credentials.json"

In [40]:
client = bigquery.Client()
table_id = "suryo-df8.practice_case_6.question4"
file_path = r"homework_csv_outputs/question4.csv"

job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.CSV, skip_leading_rows = 1, autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

In [41]:
table = client.get_table(table_id)
print(
    "Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id)
)

Loaded 5 rows and 3 columns to suryo-df8.practice_case_6.question4
